# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [64]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [3]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

# Solution using CHATGPT

In [9]:
# set up environment
## Get the Key
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API")



API key looks good so far


In [10]:
#Define the API
openai=OpenAI()

In [49]:
# CODE-SNIPPET: Paste your code here
CODE_SNIPPET = """

yield from {book.get("author") for book in books if book.get("author")}

"""

In [50]:
# here is the question; type over this to ask something new
def usr_prompt(code):
    question = """Please explain what this code does and why:\n\n"""
    question += code
    return question



In [51]:
sys_prompt = """You are a tutor whose job is to explain code. The users will provide you snippets
of code and your job is to explain how the code works. You can do the same by creating sample data 
and then show how the code changes the data line by line. Please ensure to explain how any data 
generation code you use works and then focus on the code snippet provided by the user to expand 
on the details of how the code works.Please provide your response in markdown."""
#If the customer explicitly asks you to improve the code please provide
#helpful suggestions on how the code can be made more efficient and readable

In [52]:
# Get gpt-4o-mini to answer, with streaming
def explain_code(system_prompt, code):
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages = [{"role":"system","content":sys_prompt},
                    {"role":"user","content":usr_prompt(code)}]
        )
    result=response.choices[0].message.content
    display(Markdown(result))


In [48]:
explain_code(sys_prompt, CODE_SNIPPET)

Certainly! Let's break down the code snippet you provided:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### What does this code do?

1. **Set Comprehension**:
   - The expression inside the `{...}` is a *set comprehension*. A set comprehension is a concise way to create a set in Python. 
   - In this case, it iterates through a collection called `books`, retrieves the value of the `"author"` key for each `book`, and includes it in the set if it exists.

2. **Filtering**:
   - The condition `if book.get("author")` ensures that only books which have an author will be included in the resulting set. Using `book.get("author")` retrieves the value associated with the key `"author"`, and the if-condition filters out any books where this value is `None` or empty.

3. **Yielding Values**:
   - The `yield from` statement is used in a generator function context. It allows the generator to yield all values from the set produced by the comprehension one at a time.
   - This means that the function is not returning a complete set at once; instead, it allows iteration over each item of the set.

### Why use this code construct?

- **Uniqueness**: By using a set comprehension, this code guarantees that the authors yielded are unique since sets inherently do not allow duplicate values.
- **Lazy Evaluation**: Utilizing `yield from` allows for lazy evaluation of the results. This means that the items can be processed one at a time as they are required, which is useful for handling large datasets efficiently.
- **Clarity and Conciseness**: The code is compact and clear, succinctly conveying the intention to collect unique authors from a potentially large list of books.

### Sample Data to Illustrate

To show how this code works with actual data, let's create a sample list of dictionaries representing books:

```python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": ""},
    {"title": "Book D", "author": "Author 1"},
    {"title": "Book E", "author": None},
    {"title": "Book F", "author": "Author 3"},
]

# This is where we would use the generator function that includes the yield from line
def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Creating a generator
unique_authors = get_unique_authors(books)

# To display the unique authors
for author in unique_authors:
    print(author)
```

### How the Data is Processed

Let's walk through the comprehension and see how it generates the output:

1. **Input List**: 

   The `books` list has 6 dictionaries.

2. **Processing Each Book**:

   - `book = {"title": "Book A", "author": "Author 1"}`: `book.get("author")` returns "Author 1", which is added to the set.
   - `book = {"title": "Book B", "author": "Author 2"}`: `book.get("author")` returns "Author 2", which is added to the set.
   - `book = {"title": "Book C", "author": ""}`: `book.get("author")` returns "", which fails the filter condition and is ignored.
   - `book = {"title": "Book D", "author": "Author 1"}`: `book.get("author")` returns "Author 1" again, but it's already in the set, so it's ignored.
   - `book = {"title": "Book E", "author": None}`: `book.get("author")` returns `None`, which also fails the filter condition and is ignored.
   - `book = {"title": "Book F", "author": "Author 3"}`: `book.get("author")` returns "Author 3", which is added to the set.

3. **Final Set**: After processing all books, the resulting set would be `{"Author 1", "Author 2", "Author 3"}`.

4. **Output**: The generator will now yield these unique authors one by one when iterated in a loop.

### Conclusion

In summary, the given code snippet uses a combination of set comprehension and yielding to efficiently create and return unique authors from a list of books. The use of filtering eliminates any blank or missing author entries, ensuring the final result is meaningful.

# Streaming solution

In [53]:
#Creating a streaming solution
def explain_code_stream(system_prompt, code):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages = [{"role":"system","content":sys_prompt},
                    {"role":"user","content":usr_prompt(code)}],
        stream=True
        )
    response= ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [54]:
explain_code_stream(sys_prompt, CODE_SNIPPET)

Certainly! Let's break down this code snippet step by step to understand what it does and how it works.

### Explanation of Code

1. **Context**: 
   This snippet is using a generator to yield values from a set comprehension. Specifically, it retrieves the authors of books in a collection named `books`, which is likely a list (or another iterable) of dictionaries.

2. **`{book.get("author") for book in books if book.get("author")}`**:
   - This part is called a **set comprehension**. It's creating a set of unique author values from the `books` collection.
   - `book` is a variable that iterates through each item in `books`.
   - `book.get("author")` is used to safely access the "author" key from each book dictionary. If the key "author" does not exist, it will return `None`, preventing a KeyError.
   - The `if book.get("author")` condition filters only those books that have a valid (non-None) author. So, only books with authors will be considered in the set.

3. **`yield from`**:
   - This is a syntax used in Python generators. It delegates yielding values to another iterable (in this case, the set created earlier) instead of handling the values in a loop or using a list.
   - As a result, each unique author found in the `books` collection will be yielded one by one whenever the generator function is called.

### Sample Data Generation

Let's create a sample list of `books` to demonstrate how this code works.

python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": None},
    {"title": "Book D", "author": "Author 1"},  # Duplicate author
    {"title": "Book E", "author": "Author 3"},
    {"title": "Book F"},  # No author key
]


### How the Code Processes the Sample Data

1. The code starts by creating a set comprehension:
   python
   {book.get("author") for book in books if book.get("author")}
   
   
   - Iterating through each book:
       - For "Book A": author is "Author 1" → Included
       - For "Book B": author is "Author 2" → Included
       - For "Book C": author is `None` → Excluded
       - For "Book D": author is "Author 1" → Duplicate (already included)
       - For "Book E": author is "Author 3" → Included
       - For "Book F": no author key → Excluded

   Resulting set: `{"Author 1", "Author 2", "Author 3"}`.

2. When using `yield from`:
   - The generator will yield each author one at a time when iterated over:
       - First, it yields `"Author 1"`.
       - Then it yields `"Author 2"`.
       - Finally, it yields `"Author 3"`.

### Conclusion

The complete line of code is useful for extracting and yielding unique author names from a list of book dictionaries, while gracefully skipping over any books that do not have an author defined. This is a clean and efficient way to obtain a collection of authors without duplicates.

In [57]:
CODE_SNIPPET_NEW = """

[x if x >= 300 else x+10 for x in y ]

"""

In [58]:
explain_code_stream(sys_prompt, CODE_SNIPPET_NEW)

Certainly! Let's break down the given list comprehension step-by-step:

### Code Explanation

python
[x if x >= 300 else x + 10 for x in y]


This line of code is a list comprehension that generates a new list based on the values found in the iterable `y`. Let's go through the different components of the code:

1. **List Comprehension**: This is a compact way to create lists in Python. The general syntax is:
   python
   [<expression> for <item> in <iterable>]
   
   In this case, the `<expression>` is `x if x >= 300 else x + 10`.

2. **Conditional Expression**: The expression `x if x >= 300 else x + 10` is a conditional expression (also known as a ternary operator). It evaluates to:
   - `x` if the condition `x >= 300` is `True`
   - `x + 10` if the condition is `False`

3. **Iterating Over `y`**: The list comprehension iterates over each element `x` in the iterable `y`.

### Working of the Code

To understand how this works, let’s create sample data for `y` and see how the code modifies that data.

#### Sample Data

python
y = [250, 300, 280, 320, 290]


Now we apply the list comprehension to this data:

### Step-by-Step Transformation

1. **Element: 250**
   - Condition: `250 >= 300` → `False`
   - Result: `250 + 10` → `260`

2. **Element: 300**
   - Condition: `300 >= 300` → `True`
   - Result: `300`

3. **Element: 280**
   - Condition: `280 >= 300` → `False`
   - Result: `280 + 10` → `290`

4. **Element: 320**
   - Condition: `320 >= 300` → `True`
   - Result: `320`

5. **Element: 290**
   - Condition: `290 >= 300` → `False`
   - Result: `290 + 10` → `300`

### Final Result

After processing each element, the new list created by the list comprehension would be:

python
[260, 300, 290, 320, 300]


### Summary

- The code generates a new list where, for each element `x` in `y`, if `x` is less than 300, it adds 10 to `x`.
- If `x` is 300 or greater, it keeps `x` unchanged.
- This is useful when you want to adjust values based on a specific threshold (in this case, 300).

Feel free to ask if you have further questions!

# Get Llama 3.2 to answer using OPENAI API

In [67]:
openai_ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [68]:
#Creating a streaming solution
def explain_code_stream_llama_openai(client, system_prompt, code):
    stream = client.chat.completions.create(
        model=MODEL_LLAMA,
        messages = [{"role":"system","content":sys_prompt},
                    {"role":"user","content":usr_prompt(code)}],
        stream=True
        )
    response= ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [69]:
explain_code_stream_llama_openai(openai_ollama,sys_prompt, CODE_SNIPPET)

**Yield From Loop Explanation**
======================================

The given code snippet is a part of a Python generator expression within a `yield from` statement. Let's break it down:

### Code Snippet

python
yield from {book.get("author") for book in books if book.get("author")}


### Explanation

This line of code uses the following components to generate values:

1.  **Generator Expression**: It's an expression that consists of an iterable (like a list or a dictionary) on the left side and a `for` loop, a `if` condition, and a set comprehension on the right.
2.  **Yield From Statement**: The `yield from` statement tells Python to create a new iterator for each item yielded by the generator expression.

### How it Works (with Sample Data)

Assume we have a dictionary `books` with multiple books, where each book has an 'author' key. Let's define our sample data:

python
# Sample Data: Books Dictionary
books = {
    "book1": {"title": "Book Title 1", "author": "Author A"},
    "book2": {"title": "Book Title 2", "author": "Author B"},
    "book3": {"title": "Book Title 3", "unknown": "Unknown Value"}  # Missing 'author'
}


We'll now walk through how this code generates values using our sample data:

1.  **for book in books**: This outer loop iterates over each book in the `books` dictionary.

    *   Book Count: 3

    |
    |-- Book: "book1"
    |-- Book: "book2"
    |-- Book: "book3"

2.  **if book.get("author")**: The inner condition filters books based on their 'author' property.

    *   `book.get("author")` returns `None` because `"book3"` is missing the 'author' key.
    |   If `False`, skip this iteration (not part of our sample output)
    +---------------+
    |  Book: "book1"    |
    |  (Author A present) |
    +---------------+
    |  Book: "book2"    |
    +---------------+


3.  `{book.get("author") for book in books if book.get("author")}`: This is the inner iterable.

    *   It returns a set comprehension with author values for matching books.

        +---------------+--------+
        |  Book: "book1"| Author A|
        +---------------+---------+--------------+
                  |     |
                  v     v
          {"Author A"}

    *   `"Book2"` is missing the 'author' key, so this iteration doesn't count.

4.  `yield from {book.get("author") for book in books if book.get("author")}`: This line tells Python what values to pass through as return values using `yield`.

Let's visualize an excerpt of how this `yield from` statement works with output:


# Sample Iteration:
Book: "book1"
  yield {"Author A"} 

Book: "book2" 
    yield {} (missing 'author')
Skip iteration for book3

Result set returned so far {
    {'Author A'}
}
.

This code effectively filters out books missing the `'title'` or `'author'`, and then yields only the matching author.

# Get Llama 3.2 to answer using OLLAMA API

In [72]:
#Creating a streaming solution
def explain_code_stream_llama_ollama(system_prompt, code):
    stream = ollama.chat(
        model=MODEL_LLAMA,
        messages = [{"role":"system","content":sys_prompt},
                    {"role":"user","content":usr_prompt(code)}],
        stream=True
        )
    response= ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk['message']['content'] or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [73]:
explain_code_stream_llama_ollama(sys_prompt, CODE_SNIPPET)

**Code Explanation**

This code snippet uses a combination of Python's `yield from` statement, dictionary comprehension, and list comprehension to extract the authors of books from an iterable (`books`) and return them as a generator.

### Breakdown

1. `{book.get("author") for book in books if book.get("author")}` is a dictionary comprehension:
	* It iterates over each book `book` in the `books` iterable.
	* For each book, it attempts to retrieve the value associated with the key `"author"`.
	* If the key exists (i.e., the book has an author), its value is included in the resulting iterator.
2. `yield from ...` is a Python 3.3+ feature that allows us to "yield from" an iterable, which means yielding each item from the inner iterable without creating a new scope.

### How it works

The entire expression is evaluated as follows:

1. The dictionary comprehension iterates over each book in `books`.
2. For each book, it attempts to retrieve the author.
3. If successful, the author's value is included in the iterator.
4. The `yield from` statement then yields each item from this iterator.

### Why

This code snippet seems to be designed to extract authors from a list of books. In other contexts, you might see similar code used for filtering or transforming data based on certain conditions.

Here's an example of how this code would work:

**Sample Data**
python
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Jane Smith"}
]

Running the code snippet on this data would result in a generator that yields:

`["John Doe", "Jane Smith"]`

Note that `None` values are excluded from the output.

**Expanded Example with Sample Data and Code**

Let's see how the code works line by line:
python
# Define sample books data
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Jane Smith"}
]

# Define the code snippet as a string
code_snippet = """
yield from {book.get("author") for book in books if book.get("author")}
"""

# Execute the code snippet using Python (using eval)
result_generator = eval(code_snippet)

# Print each item yielded by the generator
for author in result_generator:
    print(author)

Output:

John Doe
Jane Smith
